# LSTM con WIKI

Modelo básico con los embeddings de fasttext

In [1]:
import pandas as pd
import csv

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)

text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)

max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [3]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Wikipedia Embeddings

Me quedo sólo con los embeddings de unigramas...

In [4]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "wiki.es.vec")) as f:
    for i, line in enumerate(f):
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
            word_to_vec[word] = vec
        except:
            continue
        
        
embedding_size = len(word_to_vec["hola"])

In [5]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
embedding_matrix.shape

(200000, 300)

In [10]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, Dropout
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, weights=[embedding_matrix], trainable=False))
model.add(LSTM(100))
model.add(Dropout(0.65))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.35))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
model.fit(X_train, y_train, 
          validation_data=(X_dev, y_dev), 
          epochs=20, batch_size=32)


Train on 4500 samples, validate on 500 samples
Epoch 1/20
4500/4500 [==============================] - 7s 2ms/step - loss: 0.6585 - acc: 0.6231 - val_loss: 0.6474 - val_acc: 0.6420
Epoch 2/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.6016 - acc: 0.6949 - val_loss: 0.6126 - val_acc: 0.6720
Epoch 3/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.5588 - acc: 0.7276 - val_loss: 0.5904 - val_acc: 0.7020
Epoch 4/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.5388 - acc: 0.7431 - val_loss: 0.5892 - val_acc: 0.6880
Epoch 5/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.5158 - acc: 0.7553 - val_loss: 0.5714 - val_acc: 0.7260
Epoch 6/20
4500/4500 [==============================] - 7s 1ms/step - loss: 0.5078 - acc: 0.7607 - val_loss: 0.5908 - val_acc: 0.7080
Epoch 7/20
4500/4500 [==============================] - 6s 1ms/step - loss: 0.4962 - acc: 0.7682 - val_loss: 0.5542 - val_acc: 0.7320
Epoch 8/20
4500

In [11]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 0s 590us/step
Loss        : 0.5336
Accuracy    : 0.7540
Precision   : 0.7538
Recall      : 0.6622
F1          : 0.7050


## Bidirectional LSTM

In [12]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(LSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])

print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=25, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 30, 300)           60000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               320800    
_________________________________________________________________
dropout_7 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_8 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 129       
Total params: 60,346,657
Trainable params: 346,657
Non-trainable params: 60,000,000
__________________________________________________________

In [13]:
from hate.utils import print_evaluation

print_evaluation(model, X_dev, y_dev.values)

500/500 [==============================] - 1s 1ms/step
Loss        : 0.5199
Accuracy    : 0.7680
Precision   : 0.7345
Recall      : 0.7477
F1          : 0.7411
